In [17]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from model_handler import ModelHandler
from feature_selection import FeatureSelectionAndGeneration
handler = ModelHandler()
dataset = handler.dataset
train_set = dataset[handler.train_mask]
test_set = dataset[~handler.train_mask]

In [24]:
import random
# y_test = [random.randrange(4) for x in range(len(test_set))] -> I don't understand if you retrieve it from somewhere else

The dataset includes different risks that need a prediction. Every risk is considered as a different target of labels, namely a response variable.

The aim is to build a model able to predict each risk in the most accurate way possible. However, the learning process is different for each of them, meaning that the minimum set of variables that best explain the largest amount of variance in the dataset is unique for every risk. As a consequence, the following pipeline will be executed as much time as the number of risks in order to return as more precise predictions as possible. 

# Dataset splitting

The first step consists in splitting the dataset into training and test sets. The first will be used during the feature selection part, which is implemented using a boosted logistic regression model. This is a supervised learning approach, thus labels are needed for the regression to be carried out. In this dataset risks are assigned to only some of the cities, therefore it's wise to select as training set all the entries containing values for the given risk. All the rest will be referred to as test set, used for the classification task, since those cities will be the ones needing a prediction.

# Feature selection

When there is a highly non-linear and complex relationship between the predictors and the labels decision trees are preferable. The dataset has many different predictors and we don't know whether this relationship is linear or not.

The most robust approach among the ensemble method is `Boosting`. It allows to aggregate many decision trees, differently from `Random Forest`, and grow them sequentially, instead of using boostrap sampling like in `Bagging`. 

The procedure consists in fitting small trees to the residuals in order to slowly improve the prediction error. Generally, model that learn slowly tend to perform better. A pitfall of Boosting, however, is that it relies very much on its tuning parameters. Hence, it's important to undergo `Cross Validation` in order to select the combination returning the highest accuracy, for every target. 
For this purpose we decided to use 10-fold cross validation in such a way to speed up the tuning process, which is already slow given the amount of parameters that need to be optimized.

In [7]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
import shutil
import os
memory_dir = '.pipeline_cache.tmp'

XgBoost has as default objective function `reg:squarederror`, which corresponds to a linear regression with mean-squared error as loss function.

In [8]:
!pip install bayesian-optimization

In [25]:
from bayes_opt import BayesianOptimization
if os.path.isdir(memory_dir):
    shutil.rmtree(memory_dir)

def init_model(**model_params):
    return Pipeline([('generation_and_selection', FeatureSelectionAndGeneration(feats_num=200)), ('classifier', xgb.XGBClassifier(**model_params,use_label_encoder=False))],memory=memory_dir)
    



# def boosting_reg(train, y_train, risk, best_parameters):
    
#     '''Cross Validation'''

In [26]:
from sklearn.model_selection import cross_val_score
from data.labeled.preprocessed import RISKS_MAPPING
optimal_params = {}
CONSTANTS = {'subsample': 0.8}
for (risk, total_set, [train_set, valid_set]) in handler.get_total_train_val_set_per_risk():
    print(f"\n\n**Risk: {RISKS_MAPPING[risk]}**")
    print(f"Annotated Samples Size: {total_set.shape[0]}\n")
    def xgb_evaluate(max_depth, 
                     gamma, 
                     alpha,
                     colsample_bytree, n_estimators):
        params = {'max_depth': int(max_depth),
                  'subsample': 0.8,
                  'alpha': alpha,
                  'gamma': gamma,
                  'colsample_bytree': colsample_bytree,
                   'n_estimators': int(n_estimators),
                  "eval_metric": 'merror'}
        params.update(CONSTANTS)

        model = init_model(**params)
        train_tuple = (total_set[handler.feat_names], total_set[risk])
        reg_cv = model.fit(*train_tuple)
        cv_result = np.mean(cross_val_score(model, *train_tuple, cv=3))
        return cv_result
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (1, 7), 
                                                 'alpha': (0,20),
                                                 'gamma': (0, 1),
                                                 'colsample_bytree': (0.3, 0.9),
                                                 "n_estimators":[200,1000],
                                                }
                                  
                                 )
    
    # Use the expected improvement acquisition function to handle negative numbers
    # Optimally needs quite a few more initiation points and number of iterations
    xgb_bo.maximize(init_points=10, n_iter=10)
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params.update(CONSTANTS)
    optimal_params[risk] = params



**Risk: Higher water prices**
Annotated Samples Size: 87

|   iter    |  target   |   alpha   | colsam... |   gamma   | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.5287   |  8.261    |  0.5943   |  0.9781   |  1.554    |  320.8    |
|  2        |  0.4943   |  6.947    |  0.8533   |  0.2975   |  1.408    |  758.0    |
|  3        |  0.4713   |  3.671    |  0.6832   |  0.05666  |  5.44     |  684.4    |
|  4        |  0.5287   |  9.094    |  0.7876   |  0.7497   |  3.975    |  790.8    |
|  5        |  0.5287   |  17.54    |  0.4289   |  0.6056   |  4.397    |  445.3    |
|  6        |  0.4598   |  2.145    |  0.3887   |  0.1292   |  2.345    |  947.1    |
|  7        |  0.4828   |  5.783    |  0.6229   |  0.1129   |  2.553    |  317.0    |
|  8        |  0.5287   |  15.43    |  0.5483   |  0.1391   |  2.627    |  597.1    |
|  9        |  0.5287   |  9.841    |  0.4299   |  0.9625   |  6.449    |  813.7

|  13       |  0.7318   |  9.79     |  0.7216   |  0.3374   |  2.821    |  393.8    |
|  14       |  0.7318   |  12.23    |  0.8831   |  0.7808   |  3.342    |  641.8    |
|  15       |  0.7318   |  20.0     |  0.9      |  1.0      |  1.0      |  718.8    |
|  16       |  0.7318   |  13.44    |  0.835    |  0.5752   |  5.261    |  567.0    |
|  17       |  0.7318   |  11.65    |  0.4412   |  0.3482   |  4.168    |  900.4    |
|  18       |  0.4981   |  0.2669   |  0.843    |  0.04698  |  5.645    |  882.2    |
|  19       |  0.5019   |  0.0      |  0.5942   |  0.7009   |  1.0      |  562.1    |
|  20       |  0.7318   |  19.61    |  0.3606   |  0.5939   |  4.897    |  575.7    |


**Risk: Declining water quality**
Annotated Samples Size: 183

|   iter    |  target   |   alpha   | colsam... |   gamma   | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.541    |  8.192    |  0.7562   |  0.9169   |  2.425    |  

|  4        |  0.6926   |  2.66     |  0.877    |  0.9934   |  5.448    |  847.5    |
|  5        |  0.6926   |  4.474    |  0.303    |  0.3728   |  6.371    |  436.0    |
|  6        |  0.7078   |  8.586    |  0.4623   |  0.7262   |  1.18     |  916.3    |
|  7        |  0.5253   |  2.098    |  0.8703   |  0.03036  |  5.388    |  639.8    |
|  8        |  0.7078   |  4.846    |  0.3938   |  0.7002   |  4.126    |  897.2    |
|  9        |  0.6169   |  1.823    |  0.6433   |  0.5901   |  1.939    |  951.9    |
|  10       |  0.6926   |  3.542    |  0.7987   |  0.7036   |  2.044    |  656.8    |
|  11       |  0.7078   |  8.732    |  0.7786   |  0.4666   |  2.976    |  449.0    |
|  12       |  0.7078   |  5.072    |  0.7111   |  1.0      |  1.0      |  676.8    |
|  13       |  0.7078   |  13.92    |  0.6051   |  0.8487   |  1.752    |  870.8    |
|  14       |  0.4026   |  0.3467   |  0.7336   |  0.05579  |  1.378    |  390.6    |
|  15       |  0.4791   |  0.3694   |  0.4064   |  0.6

/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (9.224291665943925, 0.405069903776063, 0.38451774224936985, 5.632823250027073, 927.7087637209715)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", lin

|  1        |  nan      |  9.224    |  0.4051   |  0.3845   |  5.633    |  927.7    |


...

/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (12.152941308638823, 0.6331169762754207, 0.8796874153693808, 3.6370908607322554, 939.587050160504)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", li

|  9        |  nan      |  12.15    |  0.6331   |  0.8797   |  3.637    |  939.6    |


/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/bayes_opt/target_space.py", line 191, in probe
    target = self._cache[_hashable(x)]
KeyError: (11.31601818770167, 0.8752055834147827, 0.47340905064803973, 3.9628918660171752, 579.3553674878542)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/antonella/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", l

|  10       |  nan      |  11.32    |  0.8752   |  0.4734   |  3.963    |  579.4    |


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#from data.model import MODEL_BEST_PARAMS_PATH
#pd.DataFrame(optimal_params).to_csv(MODEL_BEST_PARAMS_PATH)

In [30]:
optimal_params = {'risk0': {'alpha': 8.261465084688773,
  'colsample_bytree': 0.5942798565322505,
  'gamma': 0.9780710626692427,
  'max_depth': 1,
  'n_estimators': 320,
  'subsample': 0.8},
 'risk1': {'alpha': 13.90401059454134,
  'colsample_bytree': 0.8285387334623371,
  'gamma': 0.4349182425943219,
  'max_depth': 5,
  'n_estimators': 968,
  'subsample': 0.8},
 'risk2': {'alpha': 17.27853343713799,
  'colsample_bytree': 0.588032054558853,
  'gamma': 0.0981480232590467,
  'max_depth': 6,
  'n_estimators': 666,
  'subsample': 0.8},
 'risk3': {'alpha': 8.192315865044893,
  'colsample_bytree': 0.7562427208831419,
  'gamma': 0.9169370907046851,
  'max_depth': 2,
  'n_estimators': 485,
  'subsample': 0.8},
 'risk4': {'alpha': 7.922430928147,
  'colsample_bytree': 0.807955446502681,
  'gamma': 0.7970662889259271,
  'max_depth': 4,
  'n_estimators': 950,
  'subsample': 0.8},
 'risk5': {'alpha': 13.362754510732369,
  'colsample_bytree': 0.6246517434414811,
  'gamma': 0.6516365370180226,
  'max_depth': 1,
  'n_estimators': 415,
  'subsample': 0.8}}

def classification(train_tuple, valid_set, y_test, risk, best_parameters):
    
    model = xgb.XGBClassifier(use_label_encoder=False,**best_parameters[risk])
    model.fit(*train_tuple)
    
    y_pred = model.predict(valid_set)
    predictions = [value for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print(risk, accuracy*100.0)
    return predictions

predictions = dict()
#risks = optimal_params.keys()

'Iterate on the risks and get the partitioned data before running the classification() function'
for (risk, total_set, [train_set, valid_set]) in handler.get_total_train_val_set_per_risk():
    train_tuple = (total_set[handler.feat_names], total_set[risk])
    #train, test, y_train, y_test = data_splitting(dt,risk)
    predictions[risk] = classification(train_tuple, valid_set, y_test, risk, optimal_params)

[15:58:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ValueError: Feature shape mismatch, expected: 148, got 27